# Compared with OLS

Logic: 10 Folder Prediction on self-model vs average other groups

In [1]:
%pwd

'/mnt/g/D-storage/ESG09_Article/Code'

In [2]:
%cd ..

/mnt/g/D-storage/ESG09_Article


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [30]:
from joblib import dump, load
import numpy as np
import os 
import pandas as pd
import random
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

## Load dataset and divide them into subgroup

In [4]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [5]:
Df = pd.read_parquet(Df_Filename)

In [6]:
Df.shape

(1911212, 64)

In [7]:
Df.columns

Index(['wave', 'INCOME_2', 'Cantril_ladder', 'Health_disable', 'Relative_have',
       'Living_standard_change', 'Enough_food', 'Enough_shelter',
       'Well_rested', 'Respected', 'Smile', 'Interesting_thing', 'Enjoyment',
       'Physical_pain', 'Worry', 'Sadness', 'Stress', 'Anger',
       'City_satisficied', 'Economic_change', 'Goodtime_job', 'Sat_pubtran',
       'Sat_road', 'Sat_edu', 'Sat_qualityair', 'Sat_qualitywater',
       'Sat_healthcare', 'Sat_affhouse', 'Sat_oppofriend', 'Good_minorities',
       'Good_gayles', 'Good_immigrants', 'Donated', 'Volunteer',
       'Help_stranger', 'Voice_official', 'Local_police', 'Safety_walk',
       'Stolen', 'Assualted', 'Religion_importance', 'Children_respected',
       'Children_learn', 'Women_respected', 'Sat_dealpoor', 'Sat_perserveenv',
       'Freedom_chooselife', 'Conf_military', 'Conf_judicial',
       'Conf_government', 'Conf_financial', 'Conf_honestyelections',
       'Freedom_media', 'Corruption_business', 'Corruption_governm

In [9]:
Df.sort_index().head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
367305,4,1.351047e+01,6.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,21.0,1.0,1.0,0.0,2.0,1.0,USA
367306,4,2.296780e+06,6.0,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,34.0,1.0,1.0,0.0,2.0,5.0,USA
367307,4,3.512722e+05,8.0,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,44.0,1.0,1.0,0.0,2.0,5.0,USA
367308,4,5.404188e+04,8.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,67.0,1.0,1.0,0.0,1.0,5.0,USA
367309,4,0.000000e+00,8.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,49.0,2.0,1.0,0.0,1.0,1.0,USA


In [10]:
Df = Df.sample(frac=1, random_state=42)

In [11]:
Df.head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,7.0,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
1562723,11,20170.734058,5.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
708695,6,8664.458086,4.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,8.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND


In [12]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

In [13]:
Df.isna().any().any()

False

In [14]:
Df_young = Df.loc[Df['Age']<=40, :].drop(columns=['Age'])

In [15]:
Df_middle = Df.loc[(Df['Age']>40)&(Df['Age']<=65), :].drop(columns=['Age'])

In [16]:
Df_old = Df.loc[Df['Age']>65, :].drop(columns=['Age'])

In [17]:
yy = Df_young['Cantril_ladder']
Xy = Df_young.drop(columns=['Cantril_ladder'])

In [18]:
ym = Df_middle['Cantril_ladder']
Xm = Df_middle.drop(columns=['Cantril_ladder'])

In [19]:
yo = Df_old['Cantril_ladder']
Xo = Df_old.drop(columns=['Cantril_ladder'])

In [47]:
y = Df['Cantril_ladder']
X = Df.drop(columns=['Cantril_ladder'])

## Prediction of Each Group Based on Different Models

In [70]:
ols = LinearRegression()
kf = KFold(n_splits=10, shuffle=True, random_state=42)
Xy_encoded = pd.get_dummies(Xy, drop_first=True)
scores = cross_val_score(ols, Xy_encoded, yy, cv=kf, scoring='r2')

In [71]:
scores

array([0.31047511, 0.30359478, 0.31020385, 0.30641302, 0.30875559,
       0.3062418 , 0.3089177 , 0.30988344, 0.27808987, 0.3134841 ])

In [72]:
scores.max() - scores.min()

0.03539422763774025

In [34]:
scores.mean()

0.3056059241902413

In [35]:
scores.std()

0.009530658714331806

In [38]:
ols = LinearRegression()
kf = KFold(n_splits=10, shuffle=True, random_state=42)
Xm_encoded = pd.get_dummies(Xm, drop_first=True)
scores = cross_val_score(ols, Xm_encoded, ym, cv=kf, scoring='r2')

In [39]:
scores

array([0.37195114, 0.37685847, 0.37181933, 0.37486719, 0.37441654,
       0.36868896, 0.26802381, 0.36986432, 0.36923755, 0.37630765])

In [40]:
scores.mean()

0.36220349524020556

In [41]:
scores.std()

0.03151308391892737

In [42]:
ols = LinearRegression()
kf = KFold(n_splits=10, shuffle=True, random_state=42)
Xo_encoded = pd.get_dummies(Xo, drop_first=True)
scores = cross_val_score(ols, Xo_encoded, yo, cv=kf, scoring='r2')

In [43]:
scores

array([0.38873713, 0.38803747, 0.38959006, 0.3953112 , 0.39039817,
       0.39287455, 0.38181798, 0.39254133, 0.39596217, 0.38404042])

In [44]:
scores.mean()

0.38993104677330476

In [45]:
scores.std()

0.004320476784696853

### All

In [48]:
ols = LinearRegression()
kf = KFold(n_splits=10, shuffle=True, random_state=42)
X_encoded = pd.get_dummies(X, drop_first=True)
scores = cross_val_score(ols, X_encoded, y, cv=kf, scoring='r2')

In [49]:
scores

array([0.33886987, 0.33819521, 0.33522845, 0.33606037, 0.33872882,
       0.33694698, 0.33762807, 0.33923422, 0.3413866 , 0.33945767])

In [50]:
scores.mean()

0.33817362612436974

In [51]:
scores.std()

0.0016980141290090752

In [53]:
X_encoded.shape

(1911212, 225)

In [68]:
5.15 / 33.81

0.1523217982845312

## ANN Test

In [64]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [65]:
def r2_score(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

In [55]:
X_num = X_encoded.to_numpy()
y_num = y.to_numpy()

In [59]:
X_num = X_num.astype(float)
y_num = y_num.astype(float)

In [67]:
# Create the ANN model
model = Sequential()

# Input layer + 1st hidden layer
model.add(Dense(224, input_dim=X_encoded.shape[1], activation='relu'))

# Hidden layers (2nd to 8th)
for _ in range(7):  # 7 more layers to make it 8 hidden layers total
    model.add(Dense(224, activation='relu'))

# Output layer (assuming regression, use 'linear'; for classification, use 'sigmoid' or 'softmax')
model.add(Dense(1, activation='relu'))  # For regression

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[r2_score])

# Fit the model
model.fit(X_num, y_num, epochs=100, batch_size=32*8, validation_split=0.1)

Epoch 1/100


2025-04-20 17:25:27.831235: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-04-20 17:25:28.869678: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2216c8d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-20 17:25:28.869742: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-04-20 17:25:29.548346: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-20 17:25:31.391614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2025-04-20 17:25:33.496036: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support

6720/6720 [==============================] - 54s 6ms/step - loss: 35175.6914 - r2_score: -6989.4409 - val_loss: 26.7493 - val_r2_score: -3.6739
Epoch 2/100
6720/6720 [==============================] - 46s 7ms/step - loss: 3829.9373 - r2_score: -708.2230 - val_loss: 6.7976 - val_r2_score: -0.1814
Epoch 3/100
6720/6720 [==============================] - 45s 7ms/step - loss: 250.8978 - r2_score: -46.5597 - val_loss: 4.2356 - val_r2_score: 0.2694
Epoch 4/100
6720/6720 [==============================] - 45s 7ms/step - loss: 4.2952 - r2_score: 0.2591 - val_loss: 4.3302 - val_r2_score: 0.2532
Epoch 5/100
6720/6720 [==============================] - 45s 7ms/step - loss: 8.9919 - r2_score: -0.5248 - val_loss: 5.2750 - val_r2_score: 0.0907
Epoch 6/100
6720/6720 [==============================] - 46s 7ms/step - loss: 4.6306 - r2_score: 0.2017 - val_loss: 4.3882 - val_r2_score: 0.2430
Epoch 7/100
6720/6720 [==============================] - 45s 7ms/step - loss: 5.7651 - r2_score: 0.0067 - val_loss

KeyboardInterrupt: 